In [1]:
import pandas as pd

### load the data

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_target = pd.read_csv('sample_submission.csv')

In [3]:
df_train.isna().sum()


id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [4]:
df_test.isna().sum()


id             0
keyword       26
location    1105
text           0
dtype: int64

In [5]:
df_target.isna().sum()


id        0
target    0
dtype: int64

In [6]:
df_train.keyword = df_train.keyword.fillna(df_train.keyword.mode()[0])
df_test.keyword = df_test.keyword.fillna(df_test.keyword.mode()[0])
df_train = df_train.dropna()
df_test = df_test.dropna()

In [7]:

df_train

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [8]:
df_train["clean_text"] = df_train["text"].apply(lambda x: x.lower())

In [9]:
!pip install contractions


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import contractions

In [11]:
df_train["clean_text"] = df_train["clean_text"].apply(lambda x: contractions.fix(x))

In [12]:
df_train["clean_text"]

31      @bbcmtd wholesale markets ablaze http://t.co/l...
32      we always try to bring the heavy. #metal #rt h...
33      #africanbaze: breaking news:nigeria flag set a...
34                     crying out for more! set me ablaze
35      on plus side look at the sky last night it was...
                              ...                        
7575    on the bright side i wrecked http://t.co/uea0t...
7577    @widda16 ... he is gone. you can relax. i thou...
7579    three days off from work and they have pretty ...
7580    #fx #forex #trading cramer: iger's 3 words tha...
7581    @engineshed great atmosphere at the british li...
Name: text_clean, Length: 5080, dtype: object

In [22]:
print(df_train["text"][67])
print(df_train["clean_text"][67])

'I can't have kids cuz I got in a bicycle accident &amp; split my testicles. it's impossible for me to have kids' MICHAEL YOU ARE THE FATHER
'i cannot have kids cuz i got in a bicycle accident  split my testicles. it is impossible for me to have kids' michael you are the father


In [14]:
import re 

In [15]:
def remove_URL(text):
    """
        Remove URLs from a sample string
    """
    return re.sub(r"https?://\S+|www\.\S+", "", text)



def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)


def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 


In [16]:
df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_URL(x))

df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_html(x))

df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_non_ascii(x))

In [21]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)